In [2]:
# Import required modules
import requests
import urllib3
import secrets
import pandas as pd
import polyline
import folium
from ast import literal_eval

In [3]:
# Disable insecure request warnings from urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Strava API Request
    - https://developers.strava.com/docs/reference/
    - within this repo there is a .gitignore file which ignores secrets.py this is where the strava_payload is to handle Strava API request
        - values in secrets.py for Strava API request:
            - client_id
            - client_secret
            - refresh_token
            - grant_type
            - f 
                - which is a request for json files to the API


In [4]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

# Request Strava Token
print('Requesting Strava token... \n')
res = requests.post(auth_url, data=secrets.strava_payload, verify=False)
strava_access_token = res.json()['access_token']

# Set the authorization header using the obtained access token
header = {'Authorization': 'Bearer ' + strava_access_token}

strava_requests_page_num = 1
all_activities = []

while True:
    # Prepare the parameters for paginated request
    strava_param = {'per_page' : 15, 'page' : strava_requests_page_num}
    # Send GET request to retrieve Strava activity data
    strava_dataset = requests.get(activities_url, headers=header, params=strava_param).json()

    if len(strava_dataset) == 0:
        print('breaking out of Strava while loop because the response is zero, indicating no more activities.')
        break

    if all_activities:
        print('all activities is populated')
        all_activities.extend(strava_dataset)

    else:
        print('all activities is NOT populated')
        all_activities = strava_dataset

    strava_requests_page_num += 1

print('Total Activities: ', len(all_activities))

Requesting Strava token... 

all activities is NOT populated
all activities is populated
all activities is populated
all activities is populated
breaking out of Strava while loop because the response is zero, indicating no more activities.
Total Activities:  54


## Creating pandas dataframe for all activites from Strava api
    - Contains all activites
        - Running
        - Walking
        - Hiking
        - Biking

In [5]:
all_strava_activites = pd.DataFrame(data=all_activities)

# Inspecting and Cleaning Activities Data
    - Where can data be cleaned?
    - Many Columns are not filled with information because there is no use of a watch or heart monitor.
    - Not intersted in the social information of Strava
        - ie, photos, kudos
    - location_city, location_state actually contain no information
        - We can get location information from Google Polyline information found in the 'map' column


In [6]:
all_strava_activites.head()

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_watts,kilojoules,device_watts
0,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Sore Heel Loop and Drive By Loop,8693.2,2864,3001,317.7,Run,Run,0.0,...,10170403908,71247EE3-700D-4881-87D0-B8B996F29BBD-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
1,2,"{'id': 8586088, 'resource_state': 1}",RRG : Rush Trail to Gray’s Arch Loop,6312.2,2374,2448,173.8,Run,Run,0.0,...,10150965476,3A315073-851C-4478-8D23-F4D440BAA704-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
2,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Lode Loop and Drive By Loop,5532.4,1884,1967,124.0,Run,Run,0.0,...,10143932040,7D43355C-FECB-473B-8263-E7C4BDB870DE-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
3,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Lode Loop and Sore Heel loop,7940.5,2861,2886,153.7,Run,Run,0.0,...,10131336663,4256B6D8-14F3-4F18-8DF3-440342D551CA-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
4,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Sore Heel Loop,7090.5,2341,2365,210.0,Run,Run,0.0,...,10124522361,0B1C68DC-4E71-4EBB-8217-F688A9582DCD-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN


In [7]:
all_strava_activites_columns = all_strava_activites.columns.to_list()
all_strava_activites_columns

['resource_state',
 'athlete',
 'name',
 'distance',
 'moving_time',
 'elapsed_time',
 'total_elevation_gain',
 'type',
 'sport_type',
 'workout_type',
 'id',
 'start_date',
 'start_date_local',
 'timezone',
 'utc_offset',
 'location_city',
 'location_state',
 'location_country',
 'achievement_count',
 'kudos_count',
 'comment_count',
 'athlete_count',
 'photo_count',
 'map',
 'trainer',
 'commute',
 'manual',
 'private',
 'visibility',
 'flagged',
 'gear_id',
 'start_latlng',
 'end_latlng',
 'average_speed',
 'max_speed',
 'has_heartrate',
 'heartrate_opt_out',
 'display_hide_heartrate_option',
 'elev_high',
 'elev_low',
 'upload_id',
 'upload_id_str',
 'external_id',
 'from_accepted_tag',
 'pr_count',
 'total_photo_count',
 'has_kudoed',
 'suffer_score',
 'average_watts',
 'kilojoules',
 'device_watts']

In [8]:
# Define the columns to drop from the DataFrame
columns_to_drop = ['athlete',
                   'resource_state', 
                   'sport_type', 
                   'workout_type',
                   'location_city',
                   'location_state',
                   'location_country', 
                   'kudos_count', 
                   'comment_count', 
                   'athlete_count', 
                   'photo_count', 
                   'trainer', 
                   'commute', 
                   'manual', 
                   'private',
                   'visibility', 
                   'flagged', 
                   'gear_id', 
                   'has_heartrate', 
                   'heartrate_opt_out', 
                   'display_hide_heartrate_option', 
                   'from_accepted_tag', 
                   'total_photo_count', 
                   'has_kudoed', 
                   'average_watts', 
                   'kilojoules',
                   'achievement_count',
                   'device_watts',
                   'upload_id_str',
                   'upload_id',
                   'external_id', 
                   'suffer_score']

# Drop the specified columns from the DataFrame
all_strava_activites.drop(columns=columns_to_drop, inplace=True)

columns = all_strava_activites.columns.to_list()

In [9]:
columns

['name',
 'distance',
 'moving_time',
 'elapsed_time',
 'total_elevation_gain',
 'type',
 'id',
 'start_date',
 'start_date_local',
 'timezone',
 'utc_offset',
 'map',
 'start_latlng',
 'end_latlng',
 'average_speed',
 'max_speed',
 'elev_high',
 'elev_low',
 'pr_count']

# Creating a Pandas Dataframe for just the activity of Running
    - Filter data with the 'type' is equal to 'Run'

In [10]:
run_data = all_strava_activites.loc[all_strava_activites['type'] == 'Run']

# Reset the index of the DataFrame after filtering
run_data.reset_index(drop=True, inplace=True)

## Conversions for Metrics
    - Calculate miles, minutes, and hours
    - 'distance' is in meters
    - 'moving_time' is in seconds

In [11]:
# Calculate and add new columns, 'distance_miles', 'moving_time_minutes', and 'moving_time_hours, rounded to 2 decimal places
run_data['distance_miles'] = round(run_data['distance'] * 0.00062137119, 2)
run_data['moving_time_minutes'] = round(run_data['moving_time'] / 60, 2)
run_data['moving_time_hours'] = round(run_data['moving_time'] / 3600, 2)

/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_11480/1029525087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_data['distance_miles'] = round(run_data['distance'] * 0.00062137119, 2)
/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_11480/1029525087.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_data['moving_time_minutes'] = round(run_data['moving_time'] / 60, 2)
/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_11480/1029525087.py:4: SettingWithCopyWarning: 


In [12]:
# Calculate averages for miles and time
average_distance_miles = round(run_data['distance_miles'].mean(), 2)
print("Average Distance (miles):", average_distance_miles)
average_time_minutes = round(run_data['moving_time_minutes'].mean(), 2)
print("Average Time Ran (minutes):", average_time_minutes)

# Calculate distance for longest run
max_distance_ran = round(run_data['distance_miles'].max(), 2)
print("Longest Run:", max_distance_ran, "miles")

# Calculate total time ran
max_duration_mintues = round(run_data['moving_time_minutes'].max(), 2)
max_duration_hours = round(run_data['moving_time_hours'].max(), 2)
print("Longest Duration:", max_duration_mintues,"minutes. Converted to hours:", max_duration_hours)

# Calculate total miles ran
total_distance_miles = round(run_data['distance'].sum() * 0.00062137119, 2)
print("Total Distance Covered to the date (miles):", total_distance_miles)

Average Distance (miles): 4.02
Average Time Ran (minutes): 40.74
Longest Run: 9.62 miles
Longest Duration: 117.57 minutes. Converted to hours: 1.96
Total Distance Covered to the date (miles): 152.58


# Retrieve and Decode Polyline for mapping
    - Google Polyline information:
        - https://developers.google.com/maps/documentation/utilities/polylineutility
    - When Polyline is decoded it outputs longitude and latitude listings for activity.
    - Use of polyline module
        - https://pypi.org/project/polyline/

In [13]:
# Create a new DataFrame 'all_run_map_data' form the 'map' column in 'run_data' for polyline data
all_run_map_data = pd.DataFrame(run_data['map'].to_list())

# Remove the first character 'a' from the 'id' column to match id's between two DataFrames
all_run_map_data['id'] = all_run_map_data['id'].str.slice(start=1)

# Drop the 'map' column from 'run_data'
run_data.drop(columns='map', inplace=True)

/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_11480/1717030472.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_data.drop(columns='map', inplace=True)


In [14]:
# Create a new DataFrame 'decoded_df_all' with columns 'id' and 'decoded_polyline'
decoded_df_all = pd.DataFrame(columns=['id', 'decoded_polyline'])

# Iterate over each row in 'all_run_map_data'
for index, row in all_run_map_data.iterrows():
    polyline_str = row['summary_polyline']

    # Decode the polyline string using 'polyline.decode()'
    decoded_polyline = polyline.decode(polyline_str)

    # Append the decoded polyline and its corresponding ID to 'decoded_df_all'
    decoded_df_all = decoded_df_all.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=True)

/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_11480/1664071840.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decoded_df_all = decoded_df_all.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=True)
/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_11480/1664071840.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decoded_df_all = decoded_df_all.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=True)
/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_11480/1664071840.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decoded_df_all = decoded_df_all.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=T

In [15]:
# Print the data type of the 'id' column in both DataFrames
print('ID column datatype in Run Data: ', run_data['id'].dtype)
print('ID column datatype in Decoded Data: ', decoded_df_all['id'].dtype)

# Convert the 'id' column in 'decoded_df_all' to integer data type
decoded_df_all['id'] = decoded_df_all['id'].astype(int)

ID column datatype in Run Data:  int64
ID column datatype in Decoded Data:  object


In [16]:
# Merge the 'run_data' with 'decoded_df_all' on the 'id' column
run_data = pd.merge(run_data, decoded_df_all, on='id')

# Going further into the data to only collect runs from RRGCC land
    - For this particular project I want to focus on RRGCC owned and operated land
    - Highlight the running opportunites on climber owned land
        - Showcase different loops and routes

In [17]:
# Filter the run data for RRGCC land and different running loops
pmrp_run_data = run_data[run_data['name'].str.contains('PMRP', case=False, na=False)]
rrg_run_data = run_data[run_data['name'].str.contains('RRG', case=False, na=False)]
sore_heel_data = run_data[run_data['name'].str.contains('Sore Heel', case=False, na=False)]
lode_loop_data = run_data[run_data['name'].str.contains('Lode Loop', case=False, na=False)]
drive_by_loop_data = run_data[run_data['name'].str.contains('Drive By Loop', case=False,na=False)]

In [18]:
# Reset the index of the DataFrame after filtering
pmrp_run_data.reset_index(drop=True, inplace=True)
rrg_run_data.reset_index(drop=True, inplace=True)
sore_heel_data.reset_index(drop=True, inplace=True)
lode_loop_data.reset_index(drop=True, inplace=True)
drive_by_loop_data.reset_index(drop=True, inplace=True)

In [19]:
# Set csv path and save csv files
all_csv_path = 'csv/run/run_data.csv'
pmrp_csv_path = 'csv/run/pmrp_run_data.csv'
rrg_csv_path = 'csv/run/rrg_run_data.csv'
sore_heel_csv_path = 'csv/run/sore_heel_data.csv'
lode_loop_csv_path = 'csv/run/lode_loop_data.csv'
drive_by_csv_path = 'csv/run/drive_by_data.csv'

run_data.to_csv(all_csv_path, index=False)
pmrp_run_data.to_csv(pmrp_csv_path, index=False)
rrg_run_data.to_csv(rrg_csv_path, index=False)
sore_heel_data.to_csv(sore_heel_csv_path, index=False)
lode_loop_data.to_csv(lode_loop_csv_path, index=False)
drive_by_loop_data.to_csv(drive_by_csv_path, index=False)

In [23]:
run_data.head()

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,id,start_date,start_date_local,timezone,...,end_latlng,average_speed,max_speed,elev_high,elev_low,pr_count,distance_miles,moving_time_minutes,moving_time_hours,decoded_polyline
0,PMRP : Sore Heel Loop and Drive By Loop,8693.2,2864,3001,317.7,Run,9483406201,2023-07-19T18:29:19Z,2023-07-19T14:29:19Z,(GMT-05:00) America/New_York,...,"[37.64476018957794, -83.71412549167871]",3.035,12.572,337.0,256.4,0,5.40,47.73,0.80,"[(37.64496, -83.71436), (37.64526, -83.71463),..."
1,RRG : Rush Trail to Gray’s Arch Loop,6312.2,2374,2448,173.8,Run,9464901645,2023-07-16T20:59:54Z,2023-07-16T16:59:54Z,(GMT-05:00) America/New_York,...,"[37.80759441666305, -83.65820646286011]",2.659,6.223,392.0,255.0,0,3.92,39.57,0.66,"[(37.80793, -83.6578), (37.80796, -83.65779), ..."
2,PMRP : Lode Loop and Drive By Loop,5532.4,1884,1967,124.0,Run,9458318004,2023-07-15T21:24:26Z,2023-07-15T17:24:26Z,(GMT-05:00) America/New_York,...,"[37.64471283182502, -83.71428340673447]",2.936,10.748,338.3,259.0,0,3.44,31.40,0.52,"[(37.64481, -83.71404), (37.6448, -83.71398), ..."
3,PMRP : Lode Loop and Sore Heel loop,7940.5,2861,2886,153.7,Run,9446383475,2023-07-13T21:28:17Z,2023-07-13T17:28:17Z,(GMT-05:00) America/New_York,...,"[37.64474954456091, -83.71427284553647]",2.775,9.488,305.7,259.0,0,4.93,47.68,0.79,"[(37.64472, -83.71422), (37.64477, -83.71392),..."
4,PMRP : Sore Heel Loop,7090.5,2341,2365,210.0,Run,9439898989,2023-07-12T20:20:42Z,2023-07-12T16:20:42Z,(GMT-05:00) America/New_York,...,"[37.64480469748378, -83.71436378918588]",3.029,13.532,348.9,259.0,0,4.41,39.02,0.65,"[(37.64469, -83.71428), (37.64467, -83.71432),..."


In [21]:
lode_loop = run_data[run_data['name'].str.contains(r'\bLode Loop\b')]

In [22]:
lode_loop

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,id,start_date,start_date_local,timezone,...,end_latlng,average_speed,max_speed,elev_high,elev_low,pr_count,distance_miles,moving_time_minutes,moving_time_hours,decoded_polyline
2,PMRP : Lode Loop and Drive By Loop,5532.4,1884,1967,124.0,Run,9458318004,2023-07-15T21:24:26Z,2023-07-15T17:24:26Z,(GMT-05:00) America/New_York,...,"[37.64471283182502, -83.71428340673447]",2.936,10.748,338.3,259.0,0,3.44,31.40,0.52,"[(37.64481, -83.71404), (37.6448, -83.71398), ..."
3,PMRP : Lode Loop and Sore Heel loop,7940.5,2861,2886,153.7,Run,9446383475,2023-07-13T21:28:17Z,2023-07-13T17:28:17Z,(GMT-05:00) America/New_York,...,"[37.64474954456091, -83.71427284553647]",2.775,9.488,305.7,259.0,0,4.93,47.68,0.79,"[(37.64472, -83.71422), (37.64477, -83.71392),..."
9,PMRP : Lode Loop and Lode Hill,2615.8,843,849,96.5,Run,9059541871,2023-05-12T19:53:27Z,2023-05-12T15:53:27Z,(GMT-05:00) America/New_York,...,"[37.644639406353235, -83.71410939842463]",3.103,7.168,339.3,267.6,0,1.63,14.05,0.23,"[(37.64475, -83.71381), (37.64476, -83.71379),..."
10,PMRP : Lode Loop and Flat Holler Loop,9653.3,3349,3385,453.1,Run,9017048707,2023-05-05T18:39:29Z,2023-05-05T14:39:29Z,(GMT-05:00) America/New_York,...,"[37.644784580916166, -83.7140276748687]",2.882,14.712,332.0,259.2,1,6.00,55.82,0.93,"[(37.64486, -83.71436), (37.64513, -83.71448),..."
11,PMRP : Drive By Loop and Lode Loop,5774.7,1769,1827,131.6,Run,8973585450,2023-04-28T18:23:12Z,2023-04-28T14:23:12Z,(GMT-05:00) America/New_York,...,"[37.644623816013336, -83.7140057142824]",3.264,11.087,333.2,258.4,1,3.59,29.48,0.49,"[(37.64465, -83.71402), (37.64465, -83.71409),..."
29,PMRP : Lode Loop,2342.4,825,849,34.6,Run,8441005412,2023-01-23T21:54:18Z,2023-01-23T16:54:18Z,(GMT-05:00) America/New_York,...,"[37.64696127735078, -83.71762493625283]",2.839,11.187,293.2,262.9,0,1.46,13.75,0.23,"[(37.64693, -83.71752), (37.64695, -83.71754),..."
